# Bright Data SDK v2.1.0 Release Test

This notebook verifies that the v2.1.0 release is working correctly.

## 1. Install the Package

In [1]:
!pip install brightdata-sdk==2.1.0


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


## 2. Verify Version

In [2]:
import brightdata
print(f"Installed version: {brightdata.__version__}")
assert brightdata.__version__ == "2.1.0", f"Expected 2.1.0, got {brightdata.__version__}"
print("Version check passed!")

Installed version: 2.1.0
Version check passed!


## 3. Verify Imports

In [3]:
from brightdata import BrightDataClient, SyncBrightDataClient
from brightdata.api.async_unblocker import AsyncUnblockerClient
print("All imports successful!")

All imports successful!


## 4. Verify Client Structure

In [4]:
# Check BrightDataClient has expected attributes
client_attrs = ['scrape', 'search', 'crawler', 'scrape_url']
for attr in client_attrs:
    assert hasattr(BrightDataClient, attr) or attr in dir(BrightDataClient), f"Missing: {attr}"
print("BrightDataClient structure verified!")

# Check SyncBrightDataClient exists
print(f"SyncBrightDataClient available: {SyncBrightDataClient is not None}")

BrightDataClient structure verified!
SyncBrightDataClient available: True


## 5. Test Async Mode Parameter (Structure Only)

In [5]:
import inspect
from brightdata.api.web_unlocker import WebUnlockerService

# Check WebUnlocker has mode parameter
sig = inspect.signature(WebUnlockerService.scrape)
params = list(sig.parameters.keys())
print(f"WebUnlockerService.scrape parameters: {params}")
assert 'mode' in params, "Missing 'mode' parameter in WebUnlockerService.scrape"
print("WebUnlocker async mode parameter verified!")

WebUnlockerService.scrape parameters: ['self', 'url', 'zone', 'country', 'response_format', 'method', 'timeout', 'mode', 'poll_interval', 'poll_timeout']
WebUnlocker async mode parameter verified!


## 6. Test AsyncUnblockerClient Structure

In [10]:
# Verify AsyncUnblockerClient has the expected methods
expected_methods = ['trigger', 'get_status', 'fetch_result']
for method in expected_methods:
    assert hasattr(AsyncUnblockerClient, method), f"Missing method: {method}"
print(f"AsyncUnblockerClient methods verified: {expected_methods}")

AsyncUnblockerClient methods verified: ['trigger', 'get_status', 'fetch_result']


## 7. Live Test (Requires API Token)

Set your API token to run live tests:

In [11]:
import os

# Set your token here or via environment variable
# os.environ["BRIGHTDATA_API_TOKEN"] = "your_token_here"

TOKEN = os.environ.get("BRIGHTDATA_API_TOKEN")
if TOKEN:
    print(f"Token found: {TOKEN[:10]}...{TOKEN[-5:]}")
else:
    print("No token found. Set BRIGHTDATA_API_TOKEN to run live tests.")

Token found: 7011787d-2...d3336


### 7.1 Test SERP Sync Mode

In [12]:

async def test_serp_sync():
    if not TOKEN:
        print("Skipping: No token")
        return
    
    async with BrightDataClient(token=TOKEN) as client:
        result = await client.search.google(query="python programming", num_results=5)
        print(f"SERP sync mode - Results: {len(result.data)} items")
        print(f"First result: {result.data[0].get('title', 'N/A') if result.data else 'No data'}")
        return result

await test_serp_sync()

SERP sync mode - Results: 7 items
First result: Welcome to Python.org


<SearchResult <SearchResult success cost=N/A elapsed=2461.87ms> query={'q': 'python programming', 'location': None, 'lan...>

### 7.2 Test SERP Async Mode

In [14]:
async def test_serp_async():
    if not TOKEN:
        print("Skipping: No token")
        return
    
    async with BrightDataClient(token=TOKEN) as client:
        print("Testing SERP async mode...")
        result = await client.search.google(
            query="python programming",
            mode="async",
            poll_interval=2,
            poll_timeout=200
        )
        print(f"Result type: {type(result)}")
        print(f"Result attributes: {[a for a in dir(result) if not a.startswith('_')]}")
        print(f"Result dict: {result.__dict__ if hasattr(result, '__dict__') else 'N/A'}")
        
        if result and result.data:
            print(f"SERP async mode - Results: {len(result.data)} items")
        else:
            print("SERP async mode - No data returned")
            if hasattr(result, 'error'):
                print(f"Error: {result.error}")
        return result

await test_serp_async()

Testing SERP async mode...
Result type: <class 'brightdata.models.SearchResult'>
Result attributes: ['cost', 'country', 'data', 'data_fetched_at', 'elapsed_ms', 'error', 'get_timing_breakdown', 'page', 'query', 'results_per_page', 'save_to_file', 'search_engine', 'success', 'to_dict', 'to_json', 'total_found', 'trigger_sent_at']
Result dict: {'success': False, 'cost': None, 'error': 'Polling timeout after 200s (response_id: s4w10t1767802054612rt2phhuenppg)', 'trigger_sent_at': datetime.datetime(2026, 1, 7, 16, 7, 34, 169001, tzinfo=datetime.timezone.utc), 'data_fetched_at': datetime.datetime(2026, 1, 7, 16, 10, 55, 588762, tzinfo=datetime.timezone.utc), 'query': {'q': 'python programming'}, 'data': None, 'total_found': None, 'search_engine': 'google', 'country': None, 'page': None, 'results_per_page': None}
SERP async mode - No data returned
Error: Polling timeout after 200s (response_id: s4w10t1767802054612rt2phhuenppg)


<SearchResult <SearchResult error cost=N/A elapsed=201419.76ms> query={'q': 'python programming'}>

### 7.3 Test Web Unlocker Sync Mode

In [ ]:
async def test_web_unlocker_sync():
    if not TOKEN:
        print("Skipping: No token")
        return
    
    async with BrightDataClient(token=TOKEN) as client:
        result = await client.scrape_url(url="https://example.com")
        print(f"Web Unlocker sync mode - Status: {result.status}")
        print(f"Data length: {len(result.data) if result.data else 0} chars")
        return result

await test_web_unlocker_sync()

### 7.4 Test Web Unlocker Async Mode

In [ ]:
async def test_web_unlocker_async():
    if not TOKEN:
        print("Skipping: No token")
        return
    
    async with BrightDataClient(token=TOKEN) as client:
        print("Testing Web Unlocker async mode (may take ~2 minutes)...")
        result = await client.scrape_url(
            url="https://example.com",
            mode="async",
            poll_interval=5,
            poll_timeout=180
        )
        print(f"Web Unlocker async mode - Status: {result.status}")
        print(f"Data length: {len(result.data) if result.data else 0} chars")
        return result

await test_web_unlocker_async()

### 7.5 Test SyncBrightDataClient

In [ ]:
def test_sync_client():
    if not TOKEN:
        print("Skipping: No token")
        return
    
    with SyncBrightDataClient(token=TOKEN) as client:
        result = client.scrape_url(url="https://example.com")
        print(f"SyncBrightDataClient - Status: {result.status}")
        print(f"Data length: {len(result.data) if result.data else 0} chars")
        return result

test_sync_client()

## 8. Summary

In [ ]:
print("="*50)
print("Bright Data SDK v2.1.0 Release Test Summary")
print("="*50)
print(f"Version: {brightdata.__version__}")
print("Imports: OK")
print("Client structure: OK")
print("Async mode parameter: OK")
print("AsyncUnblockerClient: OK")
if TOKEN:
    print("Live tests: Completed (check results above)")
else:
    print("Live tests: Skipped (no token)")
print("="*50)
print("Release verification complete!")